Building kubeflow pipeline


In [1]:
# !pip install kfp

In [5]:
%%writefile Dockerfile

FROM python:3.10-slim

WORKDIR /app

COPY . ./

RUN pip install -r requirements.txt

CMD streamlit run webapp.py

Writing Dockerfile


In [7]:
# !docker build -t aman123lug/sleep-disorder .

In [8]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

In [10]:
@kfp.dsl.component
def get_data():
    getdata_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/aman123lug/sleep-disorder',
        command=['python', 'src/dataIngestion.py'],
        )
    return getdata_component

In [11]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    data_preprocessing = kfp.dsl.ContainerOp(
        name='Data-preprocessing',
        image='docker.io/aman123lug/sleep-disorder',
        command=['python', 'src/preprocess.py'],
        )
    return data_preprocessing

In [12]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/aman123lug/sleep-disorder',
        command=['python', 'src/train.py'],
        # file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [13]:
# Let see output of component configuration
debug = True
if debug :
    training_component_vis = training()
    print(training_component_vis)

In [16]:
@kfp.dsl.pipeline(
  name="Sleep-Disorder-Prediction-by-KrishAryan",
  description="Sleep Disorder prediction"
)
def sleep():
    download_data = get_data()
    # download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    # data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    # train.execution_options.caching_strategy.max_cache_staleness = "P0D"

In [17]:
kfp.compiler.Compiler().compile(sleep, "sleep-disorder-kubeflow-pipeline-sdk.yaml")